In [4]:
# Python
import os

import numpy as np

if not os.path.isdir("temp"):
    os.mkdir("temp")
a = np.asarray([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
np.savetxt("temp/test.csv", a, delimiter=",")
with open("temp/test.csv.mtd", "w") as mtd:
    mtd.write('{ "format":"csv", "header":false, "rows":3, "cols":3 }')

In [6]:
# Python
import logging

from systemds.context import SystemDSContext

# Create a federated matrix
# Indicate the dimensions of the data:
# Here the first list in the tuple is the top left Coordinate,
# and the second the bottom left coordinate.
# It is ordered as [col,row].
dims = ([0, 0], [3, 3])

# Specify the address + file path from worker:
address = "localhost:8001/temp/test.csv"

with SystemDSContext() as sds:
    fed_a = sds.federated([address], [dims])
    # Sum the federated matrix and call compute to execute
    print(fed_a.sum().compute())
    # Result should be 45.

23/11/15 17:25:17 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


45.0


In [3]:
import logging

from systemds.context import SystemDSContext
from systemds.operator.algorithm import l2svm

with SystemDSContext() as sds:
    # Generate 10 by 10 matrix with values in range 0 to 100.
    features = sds.rand(10, 10, 0, 100)
    # Add value to all cells in features
    features += 1.1
    # Generate labels of all ones and zeros
    labels = sds.rand(10, 1, 1, 1, sparsity=0.5)

    model = l2svm(features, labels, verbose=False).compute()
    logging.info(model)

23/11/15 17:20:47 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
# Python
import logging

import numpy as np
from systemds.context import SystemDSContext

addr1 = "localhost:8001/temp/test.csv"
addr2 = "localhost:8002/temp/test.csv"
addr3 = "localhost:8003/temp/test.csv"

# Create a federated matrix using two federated environments
# Note that the two federated matrices are stacked on top of each other

with SystemDSContext() as sds:
    # federated data on three locations
    fed = sds.federated([addr1, addr2, addr3], [
        ([0, 0], [3, 3]),
        ([3, 0], [6, 3]),
        ([6, 0], [9, 3])])
    # local matrix to multiply with
    loc = sds.from_numpy(np.array([
        [1, 2, 3, 4, 5, 6, 7, 8, 9],
        [1, 2, 3, 4, 5, 6, 7, 8, 9],
        [1, 2, 3, 4, 5, 6, 7, 8, 9]
    ]))
    # Multiply local and federated
    ret = loc @ fed
    # execute the lazy script and print
    print(ret.compute())

23/11/15 17:26:31 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


[[198. 243. 288.]
 [198. 243. 288.]
 [198. 243. 288.]]
